### **기본 세팅**
---

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install tqdm
!pip install catboost

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [3]:
import sys
import math
import random

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from matplotlib import font_manager, rc
rc('font', family='NanumBarunGothic') 

seed = 0

data_path = "/content/drive/Shareddrives/Parking_pred/data/Encoded/"
result_path = "/content/drive/Shareddrives/Parking_pred/jihoon/result/"
result_file = "cat_baseline.csv"

train = pd.read_excel(data_path + "train.xlsx")
test = pd.read_excel(data_path + "test.xlsx")
api_lh = pd.read_excel(data_path + "openapi_lh.xlsx")
api_mh = pd.read_excel(data_path + "openapi_mh.xlsx")
age_gender = pd.read_excel(data_path + "age_gender_info.xlsx")

submission = pd.read_excel(data_path + "sample_submission.xlsx")

train.shape, test.shape, api_lh.shape, api_mh.shape

((2952, 15), (1022, 14), (5604, 8), (13708, 10))

### **오류 데이터 처리**
---

In [4]:
train_code = ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
test_code = ['C2335', 'C1327']
submission_code = ['C2335', 'C1327', 'C2675']
for c in train_code:
    train = train[train['단지코드'] != c]

# for c in test_code:
#     test = test[test['단지코드'] != c]

# for c in submission_code:
#     submission = submission[submission['code'] != c]

train.shape, test.shape, submission.shape

((2896, 15), (1022, 14), (150, 2))

In [5]:
c2085 = train[train['단지코드'] == 'C2085'].index
train.loc[c2085, '단지코드'] = 'C1397'

c1397 = train[train['단지코드'] == 'C1397'].index
train.loc[c1397, '총세대수'] = 1339

In [6]:
c2461 = train[train['단지코드'] == 'C2431'].index
train.loc[c2461, '단지코드'] = 'C1649'

c1649 = train[train['단지코드'] == 'C1649'].index
train.loc[c1649, '총세대수'] = 1047
train.loc[c1649, '등록차량수'] = 1214

In [7]:
c2675 = test[test['단지코드'] == 'C2675']
c2675['등록차량수'] = 1279
c2675['단지코드'] = 'C1036'
c2675

# test = test[test['단지코드'] != 'C2675']

train = train.append(c2675)
train = train.reset_index(drop=True)
c1036 = train[train['단지코드'] == 'C1036'].index
train.loc[c1036, '총세대수'] = 1254

test[test['단지코드'] == 'C2675'], train[train['단지코드'] == 'C1036']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(      단지코드  총세대수  ... 도보 10분거리 내 버스정류장 수 단지내주차면수
 579  C2675   512  ...                  3    1016
 580  C2675   512  ...                  3    1016
 581  C2675   512  ...                  3    1016
 582  C2675   512  ...                  3    1016
 583  C2675   512  ...                  3    1016
 584  C2675   512  ...                  3    1016
 585  C2675   512  ...                  3    1016
 586  C2675   512  ...                  3    1016
 
 [8 rows x 14 columns],
        단지코드  총세대수 임대건물구분  ... 도보 10분거리 내 버스정류장 수 단지내주차면수  등록차량수
 1490  C1036  1254    아파트  ...                3.0    1016   1279
 1491  C1036  1254    아파트  ...                3.0    1016   1279
 1492  C1036  1254    아파트  ...                3.0    1016   1279
 1493  C1036  1254    아파트  ...                3.0    1016   1279
 1494  C1036  1254    아파트  ...                3.0    1016   1279
 1495  C1036  1254    아파트  ...                3.0    1016   1279
 1496  C1036  1254    아파트  ...                3.0    1016   1279
 289

In [8]:
def get_mod_miss_households(data):
    total = data.iloc[0]['총세대수']
    sum_hh = data['전용면적별세대수'].sum()
    miss = total - sum_hh
    prop = (data['전용면적별세대수'] / sum_hh).tolist()
    miss_prop = [int(p * miss) for p in prop]
    int_miss = miss - sum(miss_prop)
    for i in range(int_miss):
        miss_prop[(i)%len(miss_prop)] += 1

    return miss_prop

total_miss = ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804',
                'C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 
                'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 
                'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 
                'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 
                'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']

for tm in total_miss:
    if train[train['단지코드'] == tm].shape[0] == 0:
        if test[test['단지코드'] == tm].shape[0] == 0:
            print("Empty: {}".format(tm))
            continue
        else:
            mod = get_mod_miss_households(test[test['단지코드'] == tm])
            idx = test[test['단지코드'] == tm].index
            test.loc[idx, '전용면적별세대수'] += mod
            # print(test[test['단지코드'] == tm][["총세대수", "전용면적별세대수"]])
    else:
        mod = get_mod_miss_households(train[train['단지코드'] == tm])
        idx = train[train['단지코드'] == tm].index
        train.loc[idx, '전용면적별세대수'] += mod
        # print(train[train['단지코드'] == tm][["총세대수", "전용면적별세대수"]])

train.shape, test.shape

Empty: C1218
Empty: C1894


((2904, 15), (1022, 14))

In [9]:
train.reset_index(drop = True, inplace=True)
test.reset_index(drop = True, inplace=True)
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000.0,82940.0,0.0,3.0,624,205
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000.0,107130.0,0.0,3.0,624,205
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000.0,107130.0,0.0,3.0,624,205
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000.0,149760.0,0.0,3.0,624,205
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000.0,149760.0,0.0,3.0,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,C1036,1254,아파트,경기도,국민임대,46.90,50,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2900,C1036,1254,아파트,경기도,국민임대,46.90,88,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2901,C1036,1254,아파트,경기도,국민임대,47.40,30,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2902,C1036,1254,아파트,경기도,국민임대,51.93,75,9,A,43819000.0,275900.0,0.0,3.0,1016,1279


### **단지 통합**

In [10]:
tmp_train = train.copy()
tmp_test = test.copy()

tmp_train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]
tmp_test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [11]:
columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', 
           '공가수', '지하철', '버스', '단지내주차면수']
cat_features = ['임대건물구분', '지역', '공급유형']
target = '등록차량수'

In [12]:
for cf in cat_features:
    map = {}
    for i, ele in enumerate(tmp_train[cf].unique()):
        map[ele] = i
    tmp_train[cf] = tmp_train[cf].map(map)
    tmp_test[cf] = tmp_test[cf].map(map)

In [13]:
tmp_train.isnull().sum(), tmp_test.isnull().sum()

(단지코드          0
 총세대수          0
 임대건물구분        0
 지역            0
 공급유형          0
 전용면적          0
 전용면적별세대수      0
 공가수           0
 신분            0
 임대보증금       581
 임대료         590
 지하철         211
 버스            4
 단지내주차면수       0
 등록차량수         0
 dtype: int64, 단지코드          0
 총세대수          0
 임대건물구분        0
 지역            0
 공급유형          0
 전용면적          0
 전용면적별세대수      0
 공가수           0
 신분            2
 임대보증금       186
 임대료         186
 지하철          42
 버스            0
 단지내주차면수       0
 dtype: int64)

In [14]:
import statsmodels.api as sm

def display_total_parknums_summary(data, seq, fig):
    total = []
    park_nums = []
    for i, code in enumerate(data['단지코드'].unique()):
        temp = data[data['단지코드']==code]
        temp.index = range(temp.shape[0])
        total.append(temp.loc[0, "총세대수"])
        park_nums.append(temp.loc[0, "단지내주차면수"])
    regline = sm.OLS(park_nums, sm.add_constant(total)).fit().fittedvalues
    fig.add_trace( go.Scatter(x=total, y=park_nums, mode="markers", 
                              marker_color=px.colors.qualitative.Plotly[seq]), row=seq+1, col=1)
    fig.add_trace(go.Scatter(x=total, y=regline,
                            mode="lines", marker_color="black"), row=seq+1, col=1)
    fig.add_trace(go.Box(y= np.array(park_nums) / np.array(total),
                            marker_color=px.colors.qualitative.Plotly[seq]), row=seq+1, col=2)
    
fig = make_subplots(2,2, column_widths=[0.7, 0.3])
fig.update_layout(width=1000, height=1000)
display_total_parknums_summary(tmp_train, 0, fig)
display_total_parknums_summary(tmp_test, 1, fig)
fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [15]:
tmp_train = tmp_train[tmp_train['단지내주차면수']/tmp_train['총세대수'] < 1.289]
tmp_train.reset_index(drop = True, inplace=True)
tmp_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2515,545,0,0,0,33.48,276,17,A,9216000.0,82940.0,0.0,3.0,624,205
1,C2515,545,0,0,0,39.60,60,17,A,12672000.0,107130.0,0.0,3.0,624,205
2,C2515,545,0,0,0,39.60,20,17,A,12672000.0,107130.0,0.0,3.0,624,205
3,C2515,545,0,0,0,46.90,38,17,A,18433000.0,149760.0,0.0,3.0,624,205
4,C2515,545,0,0,0,46.90,19,17,A,18433000.0,149760.0,0.0,3.0,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870,C1036,1254,0,2,0,46.90,50,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2871,C1036,1254,0,2,0,46.90,88,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2872,C1036,1254,0,2,0,47.40,30,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2873,C1036,1254,0,2,0,51.93,75,9,A,43819000.0,275900.0,0.0,3.0,1016,1279


In [16]:
tmp_train['전용면적'] = tmp_train['전용면적']//3*3
tmp_test['전용면적'] = tmp_test['전용면적']//3*3

idx = tmp_train[train['전용면적']>100].index
tmp_train.loc[idx, '전용면적'] = 100
idx = tmp_test[test['전용면적']>100].index
tmp_test.loc[idx, '전용면적'] = 100

idx = tmp_train[train['전용면적']<15].index
tmp_train.loc[idx, '전용면적'] = 15
idx = tmp_test[test['전용면적']<15].index
tmp_test.loc[idx, '전용면적'] = 15

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [17]:
areas = dict.fromkeys(tmp_train['전용면적'].unique(), 0)
for i, row in tmp_train.iterrows():
    areas[row.loc['전용면적']] += row['전용면적별세대수']
areas_df = pd.DataFrame.from_dict(areas, orient='index')

fig = go.Figure(
    data=[go.Histogram(x=areas_df.index, y=areas_df[0], 
                       cumulative_enabled=True,  histnorm='percent', nbinsx=60)])
fig.show()

In [18]:
areas_std = [0.0, 33.0, 54.0, 72.0]
area_columns = ['면적_' + str(a) for a in areas_std]

In [19]:
tmp_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2515,545,0,0,0,33.0,276,17,A,9216000.0,82940.0,0.0,3.0,624,205
1,C2515,545,0,0,0,39.0,60,17,A,12672000.0,107130.0,0.0,3.0,624,205
2,C2515,545,0,0,0,39.0,20,17,A,12672000.0,107130.0,0.0,3.0,624,205
3,C2515,545,0,0,0,45.0,38,17,A,18433000.0,149760.0,0.0,3.0,624,205
4,C2515,545,0,0,0,45.0,19,17,A,18433000.0,149760.0,0.0,3.0,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870,C1036,1254,0,2,0,45.0,50,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2871,C1036,1254,0,2,0,45.0,88,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2872,C1036,1254,0,2,0,45.0,30,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2873,C1036,1254,0,2,0,51.0,75,9,A,43819000.0,275900.0,0.0,3.0,1016,1279


In [20]:
tmp_test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수
0,C1072,754,0,2,0,39.0,116,14,H,22830000.0,189840.0,0.0,2,683
1,C1072,754,0,2,0,45.0,30,14,A,36048000.0,249930.0,0.0,2,683
2,C1072,754,0,2,0,45.0,112,14,H,36048000.0,249930.0,0.0,2,683
3,C1072,754,0,2,0,45.0,120,14,H,36048000.0,249930.0,0.0,2,683
4,C1072,754,0,2,0,51.0,60,14,H,43497000.0,296780.0,0.0,2,683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,0,0,8,36.0,126,38,L,NaN,NaN,0.0,1,467
1018,C2189,382,0,3,0,27.0,98,45,H,6872000.0,106400.0,0.0,2,300
1019,C2189,382,0,3,0,27.0,20,45,H,6872000.0,106400.0,0.0,2,300
1020,C2189,382,0,3,0,39.0,204,45,H,13410000.0,144600.0,0.0,2,300


In [21]:
tmp_train.isnull().sum(), tmp_test.isnull().sum()

(단지코드          0
 총세대수          0
 임대건물구분        0
 지역            0
 공급유형          0
 전용면적          0
 전용면적별세대수      0
 공가수           0
 신분            0
 임대보증금       581
 임대료         590
 지하철         211
 버스            4
 단지내주차면수       0
 등록차량수         0
 dtype: int64, 단지코드          0
 총세대수          0
 임대건물구분        0
 지역            0
 공급유형          0
 전용면적          0
 전용면적별세대수      0
 공가수           0
 신분            2
 임대보증금       186
 임대료         186
 지하철          42
 버스            0
 단지내주차면수       0
 dtype: int64)

In [22]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [23]:
def get_max_item(data, cat_col):
    map = dict.fromkeys(data[cat_col].unique(), 0)
    for i, row in data.iterrows():
        map[row.loc[cat_col]] += row['전용면적별세대수']
    max_item = max(map, key=map.get)
    return max_item

for i, code in tqdm(enumerate(tmp_train['단지코드'].unique())):
    temp = tmp_train[tmp_train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    new_train.loc[i, '임대건물구분'] = get_max_item(temp, '임대건물구분')
    new_train.loc[i, '공급유형'] = get_max_item(temp, '공급유형')

    for j, col in enumerate(areas_std):
        if (j == len(areas_std)-1):
            new_train.loc[i, area_columns[j]] = \
            temp[temp['전용면적']>= col]['전용면적별세대수'].sum()
        else:
            new_train.loc[i, area_columns[j]] = \
            temp[(temp['전용면적']>= col) & (temp['전용면적']< areas_std[j+1])]['전용면적별세대수'].sum()

    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(tmp_test['단지코드'].unique())):
    temp = tmp_test[tmp_test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    new_test.loc[i, '임대건물구분'] = get_max_item(temp, '임대건물구분')
    new_test.loc[i, '공급유형'] = get_max_item(temp, '공급유형')

    for j, col in enumerate(areas_std):
        if (j == len(areas_std)-1):
            new_test.loc[i, area_columns[j]] = \
            temp[temp['전용면적']>= col]['전용면적별세대수'].sum()
        else:
            new_test.loc[i, area_columns[j]] = \
            temp[(temp['전용면적']>= col) & (temp['전용면적']< areas_std[j+1])]['전용면적별세대수'].sum()

409it [00:05, 71.92it/s]
150it [00:02, 71.20it/s]


In [24]:
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,공가수,지하철,버스,단지내주차면수,면적_0.0,면적_33.0,면적_54.0,면적_72.0,등록차량수
0,C2515,545.0,0.0,0.0,0.0,17.0,0.0,3.0,624.0,0.0,545.0,0.0,0.0,205.0
1,C1407,1216.0,0.0,1.0,0.0,13.0,1.0,1.0,1285.0,390.0,826.0,0.0,0.0,1064.0
2,C1945,755.0,0.0,2.0,0.0,6.0,1.0,3.0,734.0,0.0,543.0,212.0,0.0,730.0
3,C1470,696.0,0.0,3.0,0.0,14.0,0.0,2.0,645.0,0.0,696.0,0.0,0.0,553.0
4,C1898,566.0,0.0,3.0,0.0,9.0,0.0,6.0,517.0,0.0,566.0,0.0,0.0,415.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,C2586,90.0,0.0,8.0,8.0,7.0,0.0,3.0,66.0,54.0,36.0,0.0,0.0,57.0
405,C2035,492.0,0.0,4.0,0.0,24.0,0.0,1.0,521.0,156.0,336.0,0.0,0.0,246.0
406,C2020,40.0,0.0,7.0,8.0,7.0,1.0,2.0,25.0,25.0,15.0,0.0,0.0,19.0
407,C2437,90.0,0.0,10.0,2.0,12.0,0.0,1.0,30.0,90.0,0.0,0.0,0.0,16.0


In [25]:
new_test

,단지코드,총세대수,임대건물구분,지역,공급유형,공가수,지하철,버스,단지내주차면수,면적_0.0,면적_33.0,면적_54.0,면적_72.0
0,C1072,754.0,0.0,2.0,0.0,14.0,0.0,2.0,683.0,0.0,754.0,0.0,0.0
1,C1128,1354.0,0.0,2.0,0.0,9.0,0.0,3.0,1216.0,0.0,1354.0,0.0,0.0
2,C1456,619.0,0.0,7.0,0.0,18.0,0.0,16.0,547.0,0.0,575.0,44.0,0.0
3,C1840,593.0,0.0,3.0,0.0,7.0,0.0,3.0,543.0,0.0,593.0,0.0,0.0
4,C1332,1297.0,0.0,2.0,0.0,11.0,0.0,2.0,1112.0,0.0,1297.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,349.0,0.0,8.0,0.0,17.0,0.0,4.0,270.0,114.0,235.0,0.0,0.0
146,C1266,596.0,0.0,10.0,0.0,35.0,0.0,1.0,593.0,183.0,413.0,0.0,0.0
147,C2152,120.0,0.0,4.0,2.0,9.0,0.0,1.0,40.0,66.0,54.0,0.0,0.0
148,C1267,675.0,0.0,0.0,0.0,38.0,0.0,1.0,467.0,367.0,308.0,0.0,0.0


In [26]:
new_train = new_train.fillna(0.0)
new_test = new_test.fillna(0.0)

### **학습 및 추론**
--- 


In [27]:
X_train = new_train.iloc[:, 1:-1]
Y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, 
                                                      random_state=seed, shuffle=True)

In [29]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(
        loss_function='MAE',
        n_estimators=500, 
        learning_rate=0.05, 
        random_state=seed     
    )
cat.fit(x_train, y_train, eval_set=(x_valid, y_valid), use_best_model=True)

0:	learn: 282.6498460	test: 264.6512184	best: 264.6512184 (0)	total: 48.1ms	remaining: 24s
1:	learn: 274.2024069	test: 257.5434438	best: 257.5434438 (1)	total: 49.6ms	remaining: 12.3s
2:	learn: 264.5230102	test: 249.9155732	best: 249.9155732 (2)	total: 51ms	remaining: 8.45s
3:	learn: 256.1526288	test: 243.2876205	best: 243.2876205 (3)	total: 52.6ms	remaining: 6.53s
4:	learn: 248.8689138	test: 236.8921976	best: 236.8921976 (4)	total: 54.4ms	remaining: 5.38s
5:	learn: 242.3709186	test: 231.1365263	best: 231.1365263 (5)	total: 56ms	remaining: 4.61s
6:	learn: 235.2603724	test: 225.8341054	best: 225.8341054 (6)	total: 57.7ms	remaining: 4.06s
7:	learn: 228.7962376	test: 220.0621971	best: 220.0621971 (7)	total: 59.4ms	remaining: 3.65s
8:	learn: 222.5172362	test: 214.5162889	best: 214.5162889 (8)	total: 61ms	remaining: 3.33s
9:	learn: 216.9328834	test: 209.6818896	best: 209.6818896 (9)	total: 62.6ms	remaining: 3.07s
10:	learn: 211.5816054	test: 206.2733393	best: 206.2733393 (10)	total: 64.2ms	

In [30]:
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(y_valid, cat.predict(x_valid))
score

95.54082355450305

In [31]:
pred = cat.predict(x_test)

In [32]:
submission['num'] = pred

In [33]:
# submission.to_csv(result_path + result_file, index=False)